Student Answer -> Bert encoding\
Question -> BERT encoding ------.->cross attention

In [5]:
from bert_embedding import BertEmbedding


In [6]:
from torch import nn
import torch.nn.functional as F

In [7]:
import numpy as np
import math

In [15]:
bert_abstract = """We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers.
 Unlike recent language representation models, BERT is designed to pre-train deep bidirectional representations by jointly conditioning on both left and right context in all layers.
 As a result, the pre-trained BERT representations can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial task-specific architecture modifications. 
BERT is conceptually simple and empirically powerful. 
It obtains new state-of-the-art results on eleven natural language processing tasks, including pushing the GLUE benchmark to 80.4% (7.6% absolute improvement), MultiNLI accuracy to 86.7 (5.6% absolute improvement) and the SQuAD v1.1 question answering Test F1 to 93.2 (1.5% absolute improvement), outperforming human performance by 2.0%."""

In [5]:
class Evaluator:
    def __init__(self):
        pass
    
    def getBertEncoding(self,paragraph):
        sentences = paragraph.split('.')
        bert_embedding = BertEmbedding
        result = bert_embedding(sentences)
        return result

In [6]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [16]:
student_ans = "Sky is red"
question = "What is the colour of the sky?"
reference_ans ="Sky appears blue"

In [17]:
def getBertEncoding(paragraph):
        sentences = paragraph.split('.')
        bert_embedding = BertEmbedding()
        result = bert_embedding(sentences)
        return result

In [18]:
K = getBertEncoding(student_ans)
Q = getBertEncoding(question)

In [19]:
K_values = K[0][1]
Q_values = Q[0][1]

In [20]:
K_values = np.array(K_values)
Q_values = np.array(Q_values)

In [21]:
K = torch.from_numpy(K_values)
Q = torch.from_numpy(Q_values)

In [13]:
cross_attention = attention(Q,K,K)

In [133]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

tensor([[0.7137, 0.1191, 0.1672],
        [0.0628, 0.8501, 0.0871],
        [0.1534, 0.6523, 0.1942],
        [0.2714, 0.3641, 0.3644],
        [0.1581, 0.6745, 0.1674],
        [0.1706, 0.6694, 0.1599],
        [0.5924, 0.2356, 0.1720],
        [0.1272, 0.2982, 0.5746]])

In [17]:
K_values.shape

(3, 768)

In [3]:
class FeedForwardLayer(nn.Module):
    def __init__(self,inp_dim,hid_dim,dropout = 0.1):
        super().__init__()
        self.inp_dim = inp_dim
        self.hid_sim = hid_dim
        self.hidden = nn.Linear(inp_dim,hid_dim)
        self.output = nn.Linear(hid_dim,inp_dim)
        self.relu = nn.ReLU
        self.dropout = nn.Dropout
        
    def forward(self,x):
        x = self.hidden(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.output(x)
        return x

In [4]:
class EncoderBlock(nn.Module):
    def __init__(self,Value,Query,attentionBlock,feedForwardBlock,normBlock):
        super().__init__()
        self.value = Value
        self.query = Query
        self.attentionBlock = attentionBlock
        self.feedForwardBlock = feedForwardBlock
        self.normBlock = normBlock

    def forward(self,x):

        att = self.attentionBlock(x)
        x = self.normBlock(att,x)
        ff = self.feedForwardBlock(x)
        x = normBlock(ff)

        return x

    

In [ ]:
class EncoderModule(nn.Module):
    def __init__(self,Question,Answer)

In [ ]:
class RepresentationModule(nn.Module):
    def __init__

In [11]:
def checek(a = 3,b = 2):
    return b

In [25]:
Q.shape

torch.Size([8, 768])

In [14]:
multihead_attn = nn.MultiheadAttention(embed_dim = 768, num_heads=3)

In [29]:
 multihead_attn(K,K,K)

ValueError: not enough values to unpack (expected 3, got 2)

In [57]:
Query = K.reshape(1,3,768)


In [56]:
Value = Q.reshape(1,8,768)

In [43]:
be.shape

torch.Size([1, 8, 768])

In [58]:
out,att =multihead_attn(key =Value,value = Value,query = Query)

RuntimeError: shape '[-1, 9, 256]' is invalid for input of size 6144

In [46]:
out.shape

torch.Size([1, 3, 768])